In [1]:
import pandas as pd
import pyfastx
from tqdm import tqdm

In [2]:
fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.annotation.fmted.gff"
df = pd.read_csv(fn, sep='\t', header=None)
df.columns = ['chr', 'src', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df.head()

,chr,src,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11121.0,24894.0,.,+,.,ID=ENSG00000290825.2;gene_id=ENSG00000290825.2...
1,chr1,HAVANA,transcript,11426.0,14409.0,.,+,.,ID=ENST00000832828.1;Parent=ENSG00000290825.2;...
2,chr1,HAVANA,exon,11426.0,11671.0,.,+,.,ID=exon:ENST00000832828.1:1;Parent=ENST0000083...
3,chr1,HAVANA,exon,12010.0,12227.0,.,+,.,ID=exon:ENST00000832828.1:2;Parent=ENST0000083...
4,chr1,HAVANA,exon,12613.0,12721.0,.,+,.,ID=exon:ENST00000832828.1:3;Parent=ENST0000083...


Isolate transcript ids present in the GENCODE BASIC annotation

In [13]:
def get_tid(s):
    temp = s.split(';')
    tid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'ID':
            tid = kv[1]
            break
    return tid

tids = []
for i, row in df.iterrows():
    if row['feature'] == 'transcript':
        tid = get_tid(row['attribute'])
        if not tid: break; print("error parsing transcript ID")
        tids.append(tid)
print(f'{len(tids)} transcripts loaded')

158338 transcripts loaded


In [19]:
fa = pyfastx.Fasta("/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.transcripts.fa")
tids_2 = []
for ent in fa:
    tids_2.append(ent.name.split('|')[0])

In [24]:
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.transcripts.fa"
with open(out_fn, 'w') as fh:
    tids = set(tids)
    for ent in tqdm(fa):
        temp = ent.name.split('|')[0].strip()
        if temp in tids:
            fh.write(ent.raw)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 385659/385659 [00:14<00:00, 25879.09it/s]


Also make a transcripts.fa with just the protein coding transcripts: